# A very Naïve and Time Consuming Solution, we need to improve this with Trie
#### Edit 1: Stupid Naive approach (Code deleted, it was just looping over everything maybe will take one year to solve this problem)
#### Edit 2: Peter Norvich's spell checker

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
# Required Distance
!pip install textdistance

/kaggle/input/itmo-spelling-correction-autumn-2019/evaluate.py
/kaggle/input/itmo-spelling-correction-autumn-2019/no_fix.submission.csv
/kaggle/input/itmo-spelling-correction-autumn-2019/words.csv
/kaggle/input/itmo-spelling-correction-autumn-2019/train.csv


In [2]:
import textdistance
import time
import csv
import tqdm
import operator


In [3]:
word_file = '/kaggle/input/itmo-spelling-correction-autumn-2019/words.csv'
train_file = '/kaggle/input/itmo-spelling-correction-autumn-2019/train.csv'

In [4]:
# Load Word Dict
words = {}

with open(word_file, newline='') as file:
    buffer = csv.DictReader(file)
    for r in tqdm.tqdm(buffer):
        words[r['Id']] = int(r['Freq'])


724512it [00:05, 131197.12it/s]


In [5]:
len(words)

724512

In [6]:
distance_to_keep = [1, 2]
distance_function = textdistance.levenshtein

In [7]:
# WORDS = words

# def correction(word):
#     "Most probable spelling correction for word."
#     probability_dict = {}
#     # Probability for just the word
#     for word in known([word]):
#         probability_dict[word] = WORDS[word]
    
#     # Probability for one edit
#     for word in known(edits1(word)):
#         probability_dict[word] = WORDS[word]
    
# #     # Probability for two edits
# #     for word in known(edits2(word)):
# #         probability_dict[word] = WORDS[word]
        
#     if probability_dict:
#         correction = sorted(probability_dict.items(), key=operator.itemgetter(1), reverse=True)[0][0]
#     else:
#         correction = word
#     return correction


# def known(words):
#     "The subset of `words` that appear in the dictionary of WORDS."
#     if words:
#         return set(w for w in words if w and w in WORDS)
#     return set()

        

# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'бвгджзклмнпрстфхцчшщаэыуояеёюийьъabcdefghijklmnopqrstuvwxyz'.upper()
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     return set(deletes + transposes + replaces + inserts)

# def edits2(word): 
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
WORDS = words

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word)) or [word]))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    if words:
        return set(w for w in words if w and w in WORDS)
    return set()

        

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'бвгджзклмнпрстфхцчшщаэыуояеёюийьъabcdefghijklmnopqrstuvwxyz'.upper()
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [10]:
correction('UNIZOOI')

'UNIZOO'

### Testing This

In [11]:
start_time = time.time()
print(correction('ПОРНОСПЯЩИЕA'))
print("--- %s seconds ---" % (time.time() - start_time))

ПОРНОСПЯЩИЕ
--- 0.0021860599517822266 seconds ---


It works lets try on all test set

In [16]:
train = []
with open(train_file) as file:
    buffer = csv.DictReader(file)
    for r in tqdm.tqdm(buffer):
        train.append(r['Id'])

362256it [00:02, 146554.96it/s]


In [14]:
with open('submission.csv', 'w', newline='') as file:
    buffer = csv.writer(file)
    buffer.writerow(['Id', 'Predicted'])
    for word in tqdm.tqdm(train):
        corr = correction(word)
        buffer.writerow([word, corr])

100%|██████████| 362256/362256 [04:14<00:00, 1422.39it/s]


In [15]:
pd.read_csv('submission.csv')

,Id,Predicted
0,НЕЛОВКО,НЕЛОВКО
1,UNIZOOI,UNIZOO
2,ISIC,ISI
3,DUNYADA,DUNYAYA
4,ITALWAX,ITALWAX
...,...,...
362251,ПИТЕРТУР,ПИТЕРТУР
362252,СПИНОЙ1,СПИНОЙ
362253,СПОСОБИН,СПОСОБЕН
362254,СТАРШЕКЛАССНИЦА,СТАРШЕКЛАССНИЦА
